In [29]:
import findspark
findspark.init("C:\\spark-3.1.1-bin-hadoop2.7")

from pyspark import SparkConf, SparkContext, SQLContext
sc = SparkContext.getOrCreate()
print(sc)
path = "C:\\Users\\Mirela\\PycharmProjects\\Covid_2021\\israel-covid-19-cases-by-age.csv" 
data = sc.textFile(path).cache()

<SparkContext master=local[*] appName=pyspark-shell>


In [30]:
data.take(10)

['Entity,Code,Day,weekly_cases',
 '0-19,,2020-03-21,0.0134813639968279',
 '0-19,,2020-03-28,0.0660190325138779',
 '0-19,,2020-04-04,0.130848532910389',
 '0-19,,2020-04-11,0.119349722442506',
 '0-19,,2020-04-18,0.136796193497224',
 '0-19,,2020-04-25,0.124702616970658',
 '0-19,,2020-05-02,0.0549167327517843',
 '0-19,,2020-05-09,0.0178429817605075',
 '0-19,,2020-05-16,0.00852498017446471']

In [31]:
# Filter the header row 
header = data.first()
print (header)

Entity,Code,Day,weekly_cases


In [32]:
dataWoHeader = data.filter(lambda x:x !=header) 

In [33]:
dataWoHeader.first()

'0-19,,2020-03-21,0.0134813639968279'

In [34]:
fields   = header.replace(" ","_").replace("/","_").split(",")
print(fields)

['Entity', 'Code', 'Day', 'weekly_cases']


In [35]:
from datetime import datetime 
import csv
def parseTotal_cases(row):
    row = row.split(",")
    row[3] = float(row[3])
    return (row[0],row[3])

In [36]:
# Creating a Pair RDD 
Total_casesParsed = dataWoHeader.map(parseTotal_cases)

In [37]:
Total_casesParsed.take(10)

[('0-19', 0.0134813639968279),
 ('0-19', 0.0660190325138779),
 ('0-19', 0.130848532910389),
 ('0-19', 0.119349722442506),
 ('0-19', 0.136796193497224),
 ('0-19', 0.124702616970658),
 ('0-19', 0.0549167327517843),
 ('0-19', 0.0178429817605075),
 ('0-19', 0.00852498017446471),
 ('0-19', 0.0077319587628866)]

In [38]:
# Summarizing a Pair RDD 
entityTrend = Total_casesParsed.map(lambda x: (x[0],x[1]))
counts = entityTrend.reduceByKey(lambda x, y: x + y)
                       

In [39]:
counts.take(10)

[('0-19', 57.52577319587628),
 ('20-39', 48.656539930871396),
 ('40-59', 39.39565114630111),
 ('60-79', 36.142781875658564),
 ('80+', 36.50226244343892)]

In [40]:
counts.sortBy(lambda x:-x[1]).take(5)

[('0-19', 57.52577319587628),
 ('20-39', 48.656539930871396),
 ('40-59', 39.39565114630111),
 ('80+', 36.50226244343892),
 ('60-79', 36.142781875658564)]